In [1]:

import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import glob
import splat
from popsims.relations import polynomial_relation
from splat.photometry import filterMag
import splat.empirical as spe
from tqdm import tqdm
import popsims
from popsims.plot_style import  plot_style
from astropy.coordinates import SkyCoord, Galactic
import astropy.units as u
from matplotlib.lines import Line2D
plot_style()
%matplotlib inline



Welcome to the Spex Prism Library Analysis Toolkit (SPLAT)!
If you make use of any features of this toolkit for your research, please remember to cite the SPLAT paper:

Burgasser et al. (2017, Astro. Soc. India Conf. Series 14, p. 7); Bibcode: 2017ASInC..14....7B

If you make use of any spectra or models in this toolkit, please remember to cite the original source.
Please report any errors are feature requests to our github page, https://github.com/aburgasser/splat/




In [ ]:
SURVEY_AREAS= {
       'Roman HLWAS': 2_000*u.degree**2,
       'Roman HLTDS': 20*u.degree**2 ,
}

#filenames --> redo with grisms and right exposur times
FILENAMES={'Rubin Wide': 'rubin_wide',  'Rubin 10 year': 'rubin_10year'}

#add additional selection
SURVEY_DEPTHS= {
       'Roman HLWAS': {'WFI_J129': 26.7},
       'Roman HLTDS': {'WFI_J129': 26.7},
}


linestyles={'thin disk': '-', 'thick disk': '-.', 'halo': 'dotted'}
scl={'thin disk': 1, 'thick disk': 0.12, 'halo':0.005}


selection0="WFI_Z087 < 27.0, WFI_Y016 < 27.0 and WFI_J129 < 27 and LSST_I< 24.0 and LSST_Z < 23.3 and LSST_Y < 22.1"
color_selection="rz>2 and gz>2 and zy >1"
proper_motion_cuts= 'proper_motion > 1 and parallax > 3'
selection0=selection0+ 'and '+proper_motion_cuts
selection= selection0+'and '+ color_selection 
selection10year= selection0+'and '+ color_selection 


In [2]:
def compute_vols_and_numbers(data, population, selection=None):
    
    counts={}
    pointings=list(data['volume'][population][0].keys())
    drho=(AREA/len(data['footprint'])).to(u.radian**2)
    sptgrid=data['sptgrid']
    
    for idx, spt in enumerate(sptgrid):
        scale=[df.scale.mean(), df.scale_unc.mean(), df.scale_times_model.mean()]
        sn= len(df)
        
        select='spt < {}'.format(spt, spt+0.9)
        if selection !=None:
            select= select+'and '+selection
            
        sn_c= len(df.query(select))
        
        val=sn_c*np.divide(scale[-1], sn) #fraction compared to local LF
        
        #compute selection correction by pointing
        n=0.0
        for p in pointings:
            v=data['volume'][population][idx][p]*val*drho
            n += v.value
        cnts={'number': n}
        counts.update({spt: cnts})
    fn=pd.DataFrame.from_records(counts).T
    fn['spt']=sptgrid
    return fn
